In [25]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pip install torch===1.5.0 torchvision===0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 752.0 MB 6.7 kB/s  eta 0:00:01     |████████████▌                   | 295.0 MB 45.3 MB/s eta 0:00:11     |████████████▊                   | 298.7 MB 45.3 MB/s eta 0:00:11     |███████████████████▌            | 458.1 MB 22.9 MB/s eta 0:00:13     |█████████████████████████████▎  | 687.6 MB 26.3 MB/s eta 0:00:03     |█████████████████████████████▎  | 688.5 MB 26.3 MB/s eta 0:00:03     |█████████████████████████████▉  | 700.0 MB 26.3 MB/s eta 0:00:02     |██████████████████████████████▌ | 717.8 MB 56.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.6 MB 10 kB/s s eta 0:00:01     |████                            | 829 kB 51.6 MB/s eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.7.0
    Uninstalling torch-1.7.0:
      Successfully uninstalled torch-1.7.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import Multi30k
from torchtext.data import Field ,BucketIterator
import spacy
import random
import math
import os

In [4]:
#make sure code is repeatable

SEED=2222
random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic=True

In [5]:
pip install spacy download en

ERROR: Could not find a version that satisfies the requirement en (from versions: none)
ERROR: No matching distribution found for en
Note: you may need to restart the kernel to use updated packages.


In [ ]:
'''
Used in jupyternotebook -Not pipeline
spacy_de=spacy.load('de')
spacy_en=spacy.load('en')
'''

In [6]:
#downloading spacy pipeline for english
import spacy.cli 
spacy.cli.download("en_core_web_md")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [7]:
#downloading spacy pipeline for german
import spacy.cli 
spacy.cli.download("de_core_news_sm")

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [8]:
#loading pipeline for 
spacy_de=spacy.load('de_core_news_sm')
spacy_en = spacy.load("en_core_web_sm")

In [9]:
#function for tokenizing the text
'''
We will pass data into each model for processing it. Field module from torchtext helps in tokenzing . And convets tokens into lower case.Seq model pad start and stop tokens in the front and back of sequnce.So bascially torchtext works on this an convet a sequnce into takens and pad start and stop on it.  We pass processed data for tokeninzing
'''
def process_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]
def process_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

In [10]:
Source= Field(tokenize=process_de,init_token ='<sos>',eos_token='<eos>',lower=True)
Target= Field(tokenize=process_en,init_token ='<sos>',eos_token='<eos>',lower=True)

/opt/conda/lib/python3.7/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [11]:
#Splitting multi30k dataset ,here source=german and target is English
train_data,valid_data,test_data = Multi30k.splits(exts=('.de','.en'),fields =(Source,Target))

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:00<00:00, 9.15MB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 1.44MB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 1.34MB/s]
/opt/conda/lib/python3.7/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [12]:
len(train_data),len(valid_data),len(test_data)

(29000, 1014, 1000)

In [13]:
'''
Now we will build vocabulary for tokens in each language such that each token have an index with in language and that index can be used for one hot encoding . In vocab we will add words that have atleast 2 repitation
'''
Source.build_vocab(train_data,min_freq=2)
Target.build_vocab(train_data,min_freq=2)

In [14]:
'''
Now we will create iterators, this iterators will create batches outfrom the dataset, then sort them ,pad them and sent it to the appropriate device.This done with bucket iterators, wILL RETURN batches of DATA WITH SRC AND TRG attributes. This is Converted into indexed format. It create batch in sach way that contains minimum amount of padding. collecting similar lenth of sentence together combining source and target
'''
BATCH_SIZE=128

In [15]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [16]:
device.type

'cuda'

In [21]:
#datapreprocessing is performed here,data is converted into batches and preprocess it and tokenized and padded
train_iterator,valid_iterator,test_iterator = BucketIterator.splits((train_data,valid_data,test_data),batch_size=128,device=device)

/opt/conda/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [48]:
#Encoder
'''
using single layer GRU unit,produce better when campare with RNN,Use drop out. it will take previous hidden state and current input state passed. Encoder is a subclass of nn module in pytorch so we will take 
i. Here input dim is the dimension of the source langugae(german),emb_dim is the dimension of the o/p embedding layer. hid_dim is the dimension of gru unit.

'''
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim,dropout):
        super().__init__()
        self.input_dim = input_dim
        self.emb_dim   = emb_dim
        self.hid_dim = hid_dim
        self.embedding = nn.Embedding(input_dim,emb_dim)
        #gru take embeded vector as input 
        self.rnn=nn.GRU(emb_dim,hid_dim)
        self.dropout=nn.Dropout(dropout)
        
    '''
    In forward method we will pass source with the help of embedding layer this get converted into a dense representatation. This dense represenatation(embeded representation into GRU unit).Unlike LSTM GRU doesnt return a cell state
    Embedding step
    Shape of the source is sentence length/batch size this embedded and pass into rnn we get hidden state and output tensor 
    '''
    def forward(self,src):
        embedded =self.dropout(self.embedding(src))#[sent len,batch size]
        outputs,hidden =self.rnn(embedded)#[sent len ,batch size ,emb dim]
        return hidden
        #outputs-->#[sent len ,batch size,hid dim * n directions]
        #hidden--->[n layers * n directions ,batch size ,hid dim]
        
    #hidden state is the thought vector or context vector 

In [23]:
#Decoder
'''
Decorder operation is different from the normal seq to seq models. Here decorder take context vector or thought vector grom the encoder/GRU unit. Decoder take final hiddden state from the encoder as the intial hidden state(context). Using that it will produce tokens. Inorder to produce token decorder has to depend on the memory of the context vector it has seen  in the first place and this effects the decorders capability to generate tokens after end time steps.
Here we will context vector,previous hidden state and current target token in each time step thus comparing to the orignal sequence to sequence models which generates a given output based on the previous hidden state and the current input token. We pass same context vectors on same GRU over and over again. So gru dimension looks like embedded dimension plu+ hidden dimension which comes from the current token and the context vector and the output dimension being the hidden dimension 
'''
class Decorder(nn.Module):
    def __init__(self,output_dim,emb_dim,hid_dim,dropout):
        super().__intial__()
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim=output_dim
        self.embedding=nn.Embedding(output_dim,emb_dim)
        #same context is passed hidden_dim comes from current token and context vector 
        self.rnn=nn.GRU(emb_dim + hid_dim , hid_dim)
        #We will intialize a linear layer to predict the next token  only using the top layer o the decorder hidden state , we will also pass the current token and the context vector to the linear layer
        self.out =nn,Linear(emb_dim + hid_dim*2,output_dim )
        self.dropout=nn.Dropout(dropout)
        '''
        In forward method we will concatenate current token and the context vetor before feeding it to the GRU .Also concatenate current hidden state,input token and context vector before feeding it to the linear layer  
        '''
        
        def forward(self,input,hidden,context):
            #input-->[batch size]
            #hidden-->[n layers *n directions,batch_size,hid dim]
            #context-->[n layers *n directions,batch_size,hid dim]
            '''Number of layers=No of dim=1
            '''
            #hidden-->[1,batch size,hid dim]
            #context--->[1,batch size,hid dim]
            input=input.unsqueeze(0)
            #input-->[1,batch size]
            
            embedded=self.dropout(self.embedding(input))
            #embedded-->[1,batch size ,emb dim]
            
            emb_con=torch.cat((embedded,context),dim=2)
            #emb_con--> [1,batch size, emb dim +hid dim]
            
            #
            output,hidden =self.rnn(emb_con,hidden)
            #output-->[sent len,batch size,hid dim* n directions]
            #hidden-->[n layers *n directions,batch size,hid dim]
            #output ---> [1,batch size,hid dim]
            #hidden....[1,batch_size,hid dim]
            
            #pass into linear layer
            output=torch.cat((embedded.squeeze(0),hidden.squeeze(0)),dim=1)
            #utput--->[batch size, emb dim + hid dim*2]
            prediction=self.out(output)
            return prediction,hidden
            

NameError: name 'self' is not defined

In [54]:
#creating a sequnce to sequnce class
class Seq2Seq(nn.Module):
    def __init__(self,encoder,decoder,device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        '''
        Here we are using teacher forcing .Teacher forcing is  a probabaility taht determines wheither the actual ground truth from the target has to be taken or the predication from the decorder is to be considered while predicting the next target token. if 0.25 percent from ground truth. 75 percent fromm decorder
        '''
    def forward(self,src,trg,teacher_forcing_ratio=0.5):
        #src-->[sent len,batch size]
        #trg--->[sent ean ,batch size]
        
        batch_size=trg.shape[1]
        max_len=trg.shape[0]
        target_voc_size=self.decorder.output_dim
        #this tensor holds all outputs  from decorder.this is intialized to zero
        outputs=torch.zeros(max_len,batch_size,target_voc_size).to(self.device)
        #source is transimitted to form context vector. This become for decoder
        context=self.encoder(src)
        hidden=context
        input=trg[0,:]
        #for each sentence we pass input, hiddden ,context
        for t in range(1,max_len):
            output,hidden=self.decoder(input,hidden,context)
            #storing output into zero intialized vector
            outputs[t]=output
            teacher_force=random.random() < teacher_forcing_ratio
            topl =output.max(1)[1]
            input=(trg[t] if teacher_force else topl)
            
        return outputs
            
        

In [55]:
INPUT_DIM=len(Source.vocab)
OUTPUT_DIM=len(Target.vocab)
ENC_EMB_DIM=256
DEC_EMB_DIM=256
HID_DIM=512
ENC_DROPOUT=0.5
DEC_DROPOUT=0.5

enc=Encoder(INPUT_DIM,ENC_EMB_DIM,HID_DIM,ENC_DROPOUT)
dec=Encoder(OUTPUT_DIM,DEC_EMB_DIM,HID_DIM,DEC_DROPOUT)
model=Seq2Seq(enc,dec,device).to (device)

In [56]:
model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7854, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Encoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [67]:
optimizer = optim.Adam(model.parameters())

In [60]:
'''we dont calculate loss for padded sequnce as they are not part of the actual source and target sequences .We will ignore padded sequnce'''
pad_idx=Target.vocab.stoi['<pad>']
criterion=nn.CrossEntropyLoss(ignore_index=pad_idx)

In [61]:
'''
In train function we will pass model,iterator,optimizer,criterion,clip(to avoid gradient explosion)
'''
def train(model,iterator,optimizer,criterion,clip):
    model.train()
    epoch.loss=0
    for i,batch in enumerate(iterator):
        source=batch.src
        target=batch.trg#[sentlen,batch size]
        optimizer.zero_grad()
        output=model(source,target)#[sent len,batch_size,output_dim]
        #we will remove padded values before giving it to loass function
        loss=criterion(output[1:].view(-1,output.shape[2]),target[1:].view(-1))
        #trg-->[(sent len-1)* batch size]
        #output-->[(sent len-1) * batch size,output dim]
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(),clip)
        optimizer.step()
        epoch_loss+=loss.item()
        
    return epoch_loss/len(iterator)

In [62]:
def evaluate(models,iterator,criterion):
    model.eval()
    epoch_loss=0
    with torch.no_grad():
        for i,batch in enumerate(iterator):
            source=batch.src
            target=batch.trg
            output=model(source,target,0)
            loss=criterion(output[l:].view(-1,output.shape[2]),target[1:].view(-1))
            epoch_loss+=loss.item()
    return epoch_loss/len(iterator)      




In [70]:
DIR='models'
Model_DIR=os.path.join(DIR,'seq2seq_model.pt')
N_EPOCHS=10
CLIP=10
best_loss=float('inf')

if not os.path.isdir(f'{DIR}'):
    os.makedirs(f'{DIR}')
    
    
for epoch in range(N_EPOCHS):
    train_loss=train(model,train_iterator,optimizer,criterion,CLIP)
    valid_loss=evaluate(model,valid_iterator,criterion)
    
    if valid_loss<best_loss:
        best_loss=valid_loss
        torch.save(model.state_dict(),MODEL_DIR)
    print(f'Epoch: {epoch+1:03}| Train Loss : {train_loss:3f} | Train PPL:{math.exp(train_loss) : 7.3f} | Val.Loss: {valid_loss:.3f} | Val.PPL: {math.exp(valid_loss):7.3f}')

AttributeError: 'int' object has no attribute 'loss'